<h1> Parameter Tunning with MinMax
    
    GridSearchCV for: neurons with previous parameters

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

     |████████████████████████████████| 10.1 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 104.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 96.6 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=759d8d6f204e323e6a0eea2312e449dd55ef9ec157df2225abfa957d53c0e319
  Stored in directory: /home/ec2-user/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


# Import data; set X and y; fill nan values and split in test and training  data:

In [4]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018060000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

# Fill nan values (BEFORE OR AFTER TEST, TRAIN SPLIT!!!)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


# Apply feature scaling:

In [5]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [6]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
import keras.optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Using TensorFlow backend.


# Create regressor:

In [22]:
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_neurons, n_hidden = 2, optimizer = 'RMSprop', kernel_initializer="he_normal",
    bias_initializer= initializers.Ones()):
    model = Sequential()
    model.add(Dense(output_dim = n_neurons, 
                    input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dense(output_dim = 1, 
                    activation = 'linear'))
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

# Apply Randomized tunning for the whole ANN:

In [23]:
regressor = KerasRegressor(build_fn = regressor_tunning)

# Dictionary to include the parameters
parameters = {'n_neurons': np.arange(10,100, 10)
               }

tscv = TimeSeriesSplit(n_splits = 6)

# add some early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mse', patience = 15)

rnd_search_cv = GridSearchCV(estimator = regressor,
                                   param_grid = parameters,
                                   scoring = 'neg_mean_squared_error',
                                   n_jobs = -1,
                                   cv = tscv)

rnd_search_cv.fit(X_train, y_train, batch_size = 15, epochs = 100, callbacks=[early_stopping])

RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fb9ac1776d0>, as the constructor either does not set or modifies parameter n_neurons

In [13]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_
#best_estimator = rnd_search_cv.best_estimator_
#results = rnd_search_cv.cv_results_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))
#print("the best estimator is:{}".format(best_estimator))
#print("results:".format(results))

the best parameters are:{'n_neurons': 30}
the best score is:-1940.963433457642
